In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('data/train.csv')

In [3]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
from sklearn.preprocessing import LabelEncoder

In [5]:
le = LabelEncoder()

In [6]:
train['keyword'] = train['keyword'].fillna('none') #imputing Nan 
train['keyword']

0       none
1       none
2       none
3       none
4       none
        ... 
7608    none
7609    none
7610    none
7611    none
7612    none
Name: keyword, Length: 7613, dtype: object

In [7]:
le.fit(train['keyword'])

LabelEncoder()

In [8]:
len(le.classes_) #no of classes

222

In [9]:
x0 = le.transform(train['keyword'])

In [10]:
x0

array([147, 147, 147, ..., 147, 147, 147])

In [11]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np


In [12]:
ohe = OneHotEncoder()
x0 = ohe.fit_transform(np.array(x0).reshape(-1,1))

In [13]:
x0.shape

(7613, 222)

In [14]:
x0

<7613x222 sparse matrix of type '<class 'numpy.float64'>'
	with 7613 stored elements in Compressed Sparse Row format>

In [15]:
import re 
import nltk
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sshar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sshar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
"""def remove_special_characters(text):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text"""
stop_words = nltk.corpus.stopwords.words("English")

In [17]:
def preprocess(text):
    pattern=r'[^a-zA-z0-9\s]'
    text= re.sub(pattern,'',text)
    text = text.lower()
    text = ' '.join(word for word in text.split() if word not in stop_words)
    text = WordNetLemmatizer().lemmatize(text)
    text = PorterStemmer().stem(text)
    return text

In [18]:
train['text'] = train['text'].apply(preprocess)

In [19]:
train['text']

0             deeds reason earthquake may allah forgive u
1                   forest fire near la ronge sask canada
2       residents asked shelter place notified officer...
3       13000 people receive wildfires evacuation orde...
4       got sent photo ruby alaska smoke wildfires pou...
                              ...                        
7608    two giant cranes holding bridge collapse nearb...
7609    aria_ahrary thetawniest control wild fires cal...
7610    m194 [0104 utc]5km volcano hawaii httptcozdtoy...
7611    police investigating ebike collided car little...
7612    latest homes razed northern california wildfir...
Name: text, Length: 7613, dtype: object

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
#BOW
cv = CountVectorizer(ngram_range=(1, 3),max_df=0.8,min_df=5)
msg_cv = cv.fit_transform(train['text'])

In [22]:
msg_cv.shape

(7613, 4255)

In [23]:
#TFIDF
tf = TfidfVectorizer(ngram_range=(1, 3),max_df=0.7,min_df=5)
msg_tf = tf.fit_transform(train['text'])

In [24]:
msg_tf.shape

(7613, 4255)

In [25]:
x0.shape

(7613, 222)

In [26]:
from scipy.sparse import  hstack

In [29]:
x_cv = hstack([x0,msg_cv])
x_tf = hstack([x0,msg_tf])

In [30]:
x_tf

<7613x4477 sparse matrix of type '<class 'numpy.float64'>'
	with 70561 stored elements in COOrdinate format>

In [31]:
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [32]:
lg_cv = LogisticRegression()
lg_tf = LogisticRegression()


In [33]:
lg_cv.fit(x_cv,train['target'])
lg_tf.fit(x_tf,train['target'])

LogisticRegression()

In [34]:
prediction_cv = lg_cv.predict(x_cv)
prediction_tf = lg_tf.predict(x_tf)

In [35]:
accuracy_cv = accuracy_score(prediction_cv,train['target'])
accuracy_tf = accuracy_score(prediction_tf,train['target'])

In [36]:
accuracy_cv

0.8958360698804676

In [37]:
accuracy_tf

0.8582687508209641

In [38]:
sg = SGDClassifier()

In [40]:
sg.fit(x_cv,train['target'])

SGDClassifier()

In [41]:
pred = sg.predict(x_cv)

In [42]:
acc = accuracy_score(pred,train['target'])

In [43]:
acc

0.9230264022067516

In [44]:
report = classification_report(train['target'],pred)

In [45]:
print(report)

              precision    recall  f1-score   support

           0       0.91      0.96      0.93      4342
           1       0.94      0.87      0.91      3271

    accuracy                           0.92      7613
   macro avg       0.93      0.92      0.92      7613
weighted avg       0.92      0.92      0.92      7613



In [46]:
from sklearn.svm import SVC

In [47]:
svc = SVC()

In [48]:
svc.fit(x_cv,train['target'])

SVC()

In [49]:
pred1 = svc.predict(x_cv)

In [50]:
acc1 = accuracy_score(pred1,train['target'])

In [51]:
acc1

0.9106791015368448

## Accuracy without keyword                       
logistic regression for BOW = 89%                                                                 
logistic regression for TFIDF = 86%                                                            
SGD classifier for BOW = 91.5%                                                                                                                                                                                                  
SVM for BOW = 90.0%                                                                                 

## Accuracy with keyword
logistic regression for BOW = 89%
<br>
 logistic regression for TFIDF = 85%
 <br>
SGD classifier for BOW = 92.3
<br>
SVM for BOW = 91.06%

In [52]:
cm = confusion_matrix(train['target'],pred)

In [53]:
cm

array([[4169,  173],
       [ 413, 2858]], dtype=int64)

In [54]:
test = pd.read_csv('data/test.csv')

In [55]:
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [56]:
test['text'] = test['text'].apply(preprocess)

In [57]:
test.head()

,id,keyword,location,text
0,0,NaN,NaN,happened terrible car crash
1,2,NaN,NaN,heard earthquake different cities stay safe ev...
2,3,NaN,NaN,forest fire spot pond geese fleeing across str...
3,9,NaN,NaN,apocalypse lighting spokane wildfir
4,11,NaN,NaN,typhoon soudelor kills 28 china taiwan


In [58]:
pred_cv = cv.transform(test['text'])

In [60]:
test['keyword'] = test['keyword'].fillna('none')

In [61]:
test['keyword']

0       none
1       none
2       none
3       none
4       none
        ... 
3258    none
3259    none
3260    none
3261    none
3262    none
Name: keyword, Length: 3263, dtype: object

In [67]:
x0_test = le.transform(test['keyword'])

In [64]:
x0_test

array([147, 147, 147, ..., 147, 147, 147])

In [71]:
x0_test = ohe.transform(np.array(x0_test).reshape(-1,1))

In [72]:
x0_test

<3263x222 sparse matrix of type '<class 'numpy.float64'>'
	with 3263 stored elements in Compressed Sparse Row format>

In [73]:
x_test = hstack([x0_test,pred_cv])

In [74]:
x_test

<3263x4477 sparse matrix of type '<class 'numpy.float64'>'
	with 28723 stored elements in COOrdinate format>

In [75]:
final_pred = sg.predict(x_test)

In [76]:
final_pred.shape

(3263,)

In [77]:
submission = pd.DataFrame(columns=['id','target'])

In [78]:
submission['id'] = test['id']
submission['target'] = final_pred

In [80]:
submission.to_csv('submission_30-9.csv',index = False)